In [3]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [4]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [5]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [6]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [7]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [8]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [26]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    ##################################################### repeatable!
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                    labels=tf.zeros_like(gQs3))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2, dones, dones2

In [27]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [28]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2, self.dones1, self.dones2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [29]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [30]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [31]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [32]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [33]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [34]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
                print(np.count_nonzero(dones), len(dones), np.max(rates))
                break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _, dones1, dones2 = sess.run([model.a_loss2, model.a_opt2, model.dones1, model.dones2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:1.3927 dloss:2.6308 aloss2:2.1006 exploreP:0.9983
Episode:1 meanR:20.5000 R:24.0000 rate:0.0480 aloss:1.3955 dloss:2.5845 aloss2:2.1306 exploreP:0.9959
Episode:2 meanR:19.3333 R:17.0000 rate:0.0340 aloss:1.4068 dloss:2.6085 aloss2:2.1205 exploreP:0.9943
Episode:3 meanR:17.0000 R:10.0000 rate:0.0200 aloss:1.3948 dloss:2.5875 aloss2:2.1448 exploreP:0.9933
Episode:4 meanR:20.6000 R:35.0000 rate:0.0700 aloss:1.3973 dloss:2.5724 aloss2:2.1502 exploreP:0.9899
Episode:5 meanR:23.8333 R:40.0000 rate:0.0800 aloss:1.3845 dloss:2.5424 aloss2:2.1691 exploreP:0.9859
Episode:6 meanR:23.8571 R:24.0000 rate:0.0480 aloss:1.3917 dloss:2.4997 aloss2:2.1884 exploreP:0.9836
Episode:7 meanR:22.7500 R:15.0000 rate:0.0300 aloss:1.3858 dloss:2.4955 aloss2:2.2077 exploreP:0.9821
Episode:8 meanR:21.6667 R:13.0000 rate:0.0260 aloss:1.3985 dloss:2.4825 aloss2:2.2187 exploreP:0.9809
Episode:9 meanR:22.5000 R:30.0000 rate:0.0600 aloss:1.3947 dloss:2.4602 aloss2:2.2

Episode:80 meanR:21.7160 R:12.0000 rate:0.0240 aloss:1.3895 dloss:0.9088 aloss2:5.7312 exploreP:0.8403
Episode:81 meanR:21.6341 R:15.0000 rate:0.0300 aloss:1.3916 dloss:0.8333 aloss2:6.0749 exploreP:0.8391
Episode:82 meanR:21.5181 R:12.0000 rate:0.0240 aloss:1.3880 dloss:0.8062 aloss2:6.0711 exploreP:0.8381
Episode:83 meanR:21.6905 R:36.0000 rate:0.0720 aloss:1.3907 dloss:0.8186 aloss2:6.1763 exploreP:0.8351
Episode:84 meanR:21.6235 R:16.0000 rate:0.0320 aloss:1.3842 dloss:0.8048 aloss2:6.2571 exploreP:0.8338
Episode:85 meanR:21.5698 R:17.0000 rate:0.0340 aloss:1.3830 dloss:0.7881 aloss2:6.3487 exploreP:0.8324
Episode:86 meanR:21.8161 R:43.0000 rate:0.0860 aloss:1.3861 dloss:0.8206 aloss2:6.3273 exploreP:0.8289
Episode:87 meanR:21.7159 R:13.0000 rate:0.0260 aloss:1.3843 dloss:0.8059 aloss2:6.4626 exploreP:0.8278
Episode:88 meanR:21.7191 R:22.0000 rate:0.0440 aloss:1.3861 dloss:0.7862 aloss2:6.5094 exploreP:0.8260
Episode:89 meanR:21.7222 R:22.0000 rate:0.0440 aloss:1.3898 dloss:0.7448 

Episode:159 meanR:26.7700 R:36.0000 rate:0.0720 aloss:1.3729 dloss:0.6154 aloss2:10.1705 exploreP:0.6715
Episode:160 meanR:27.1900 R:70.0000 rate:0.1400 aloss:1.3782 dloss:0.6772 aloss2:10.0951 exploreP:0.6669
Episode:161 meanR:27.2000 R:23.0000 rate:0.0460 aloss:1.3752 dloss:0.6727 aloss2:9.9412 exploreP:0.6654
Episode:162 meanR:27.5200 R:46.0000 rate:0.0920 aloss:1.3711 dloss:0.5977 aloss2:10.1259 exploreP:0.6624
Episode:163 meanR:27.3200 R:25.0000 rate:0.0500 aloss:1.3661 dloss:0.6269 aloss2:10.2216 exploreP:0.6607
Episode:164 meanR:27.8900 R:78.0000 rate:0.1560 aloss:1.3703 dloss:0.6075 aloss2:10.1471 exploreP:0.6557
Episode:165 meanR:28.1100 R:35.0000 rate:0.0700 aloss:1.3660 dloss:0.6162 aloss2:10.1211 exploreP:0.6534
Episode:166 meanR:28.4800 R:63.0000 rate:0.1260 aloss:1.3698 dloss:0.6178 aloss2:10.0938 exploreP:0.6494
Episode:167 meanR:28.5700 R:21.0000 rate:0.0420 aloss:1.3783 dloss:0.6052 aloss2:10.0267 exploreP:0.6480
Episode:168 meanR:28.7600 R:32.0000 rate:0.0640 aloss:1.

Episode:238 meanR:46.6800 R:49.0000 rate:0.0980 aloss:1.3710 dloss:0.6199 aloss2:10.0315 exploreP:0.4564
Episode:239 meanR:46.8000 R:57.0000 rate:0.1140 aloss:1.3799 dloss:0.6222 aloss2:10.0066 exploreP:0.4539
Episode:240 meanR:47.0700 R:114.0000 rate:0.2280 aloss:1.3765 dloss:0.6647 aloss2:9.9258 exploreP:0.4488
Episode:241 meanR:48.0900 R:117.0000 rate:0.2340 aloss:1.3739 dloss:0.6377 aloss2:10.0179 exploreP:0.4437
Episode:242 meanR:48.7700 R:92.0000 rate:0.1840 aloss:1.3697 dloss:0.6079 aloss2:10.0632 exploreP:0.4398
Episode:243 meanR:48.7700 R:12.0000 rate:0.0240 aloss:1.3745 dloss:0.6414 aloss2:10.0229 exploreP:0.4392
Episode:244 meanR:49.6300 R:100.0000 rate:0.2000 aloss:1.3717 dloss:0.6209 aloss2:10.0367 exploreP:0.4350
Episode:245 meanR:50.2000 R:82.0000 rate:0.1640 aloss:1.3798 dloss:0.6142 aloss2:10.0451 exploreP:0.4315
Episode:246 meanR:50.2500 R:25.0000 rate:0.0500 aloss:1.3683 dloss:0.6309 aloss2:10.0548 exploreP:0.4304
Episode:247 meanR:49.6600 R:36.0000 rate:0.0720 aloss

Episode:317 meanR:86.8100 R:67.0000 rate:0.1340 aloss:1.3729 dloss:0.7548 aloss2:9.6117 exploreP:0.2284
Episode:318 meanR:87.4600 R:77.0000 rate:0.1540 aloss:1.3741 dloss:0.6591 aloss2:9.6519 exploreP:0.2267
Episode:319 meanR:87.8200 R:59.0000 rate:0.1180 aloss:1.3866 dloss:0.6657 aloss2:9.8052 exploreP:0.2254
Episode:320 meanR:88.1200 R:121.0000 rate:0.2420 aloss:1.3761 dloss:0.7035 aloss2:9.7569 exploreP:0.2229
Episode:321 meanR:87.7100 R:60.0000 rate:0.1200 aloss:1.3864 dloss:0.6439 aloss2:9.7004 exploreP:0.2216
Episode:322 meanR:87.7200 R:59.0000 rate:0.1180 aloss:1.3837 dloss:0.7009 aloss2:9.6567 exploreP:0.2203
Episode:323 meanR:88.0400 R:46.0000 rate:0.0920 aloss:1.3864 dloss:0.6388 aloss2:9.7745 exploreP:0.2194
Episode:324 meanR:87.7200 R:41.0000 rate:0.0820 aloss:1.3827 dloss:0.6676 aloss2:9.7773 exploreP:0.2185
Episode:325 meanR:87.6500 R:43.0000 rate:0.0860 aloss:1.4166 dloss:0.6609 aloss2:9.7051 exploreP:0.2176
Episode:326 meanR:87.7200 R:50.0000 rate:0.1000 aloss:1.3798 dl

...

Episode:6333 meanR:47.5800 R:57.0000 rate:0.1140 aloss:1.1378 dloss:0.7141 aloss2:12.6411 exploreP:0.0100
Episode:6334 meanR:46.8200 R:26.0000 rate:0.0520 aloss:1.1083 dloss:0.6643 aloss2:12.3346 exploreP:0.0100
Episode:6335 meanR:46.8200 R:30.0000 rate:0.0600 aloss:1.1100 dloss:0.6812 aloss2:12.4757 exploreP:0.0100
Episode:6336 meanR:46.6300 R:44.0000 rate:0.0880 aloss:1.1350 dloss:0.7200 aloss2:12.5431 exploreP:0.0100
Episode:6337 meanR:46.4800 R:58.0000 rate:0.1160 aloss:1.1263 dloss:0.6855 aloss2:12.6934 exploreP:0.0100
Episode:6338 meanR:46.2000 R:50.0000 rate:0.1000 aloss:1.1185 dloss:0.6996 aloss2:12.7694 exploreP:0.0100
Episode:6339 meanR:46.3200 R:37.0000 rate:0.0740 aloss:1.1426 dloss:0.6893 aloss2:12.6909 exploreP:0.0100
Episode:6340 meanR:46.4100 R:41.0000 rate:0.0820 aloss:1.1677 dloss:0.6792 aloss2:12.8400 exploreP:0.0100
Episode:6341 meanR:46.2200 R:38.0000 rate:0.0760 aloss:1.1935 dloss:0.7267 aloss2:12.1805 exploreP:0.0100
Episode:6342 meanR:46.4400 R:53.0000 rate:0.10

Episode:6411 meanR:43.4300 R:32.0000 rate:0.0640 aloss:1.1224 dloss:0.6965 aloss2:12.4992 exploreP:0.0100
Episode:6412 meanR:43.2500 R:38.0000 rate:0.0760 aloss:1.1253 dloss:0.7027 aloss2:12.5510 exploreP:0.0100
Episode:6413 meanR:43.1300 R:36.0000 rate:0.0720 aloss:1.1472 dloss:0.6888 aloss2:12.5685 exploreP:0.0100
Episode:6414 meanR:43.0900 R:48.0000 rate:0.0960 aloss:1.1311 dloss:0.6609 aloss2:12.3750 exploreP:0.0100
Episode:6415 meanR:43.0300 R:43.0000 rate:0.0860 aloss:1.1365 dloss:0.6993 aloss2:12.7405 exploreP:0.0100
Episode:6416 meanR:43.0400 R:42.0000 rate:0.0840 aloss:1.1433 dloss:0.7019 aloss2:12.2476 exploreP:0.0100
Episode:6417 meanR:43.1800 R:40.0000 rate:0.0800 aloss:1.1288 dloss:0.6855 aloss2:12.3667 exploreP:0.0100
Episode:6418 meanR:43.3400 R:52.0000 rate:0.1040 aloss:1.1331 dloss:0.6743 aloss2:12.4288 exploreP:0.0100
Episode:6419 meanR:43.3900 R:50.0000 rate:0.1000 aloss:1.1319 dloss:0.7026 aloss2:12.4132 exploreP:0.0100
Episode:6420 meanR:43.4900 R:68.0000 rate:0.13

Episode:6489 meanR:52.6100 R:56.0000 rate:0.1120 aloss:1.1983 dloss:0.6692 aloss2:12.0882 exploreP:0.0100
Episode:6490 meanR:52.6600 R:51.0000 rate:0.1020 aloss:1.2189 dloss:0.7066 aloss2:12.5009 exploreP:0.0100
Episode:6491 meanR:52.8200 R:62.0000 rate:0.1240 aloss:1.2003 dloss:0.7072 aloss2:12.0485 exploreP:0.0100
Episode:6492 meanR:53.7200 R:118.0000 rate:0.2360 aloss:1.1879 dloss:0.6616 aloss2:12.4174 exploreP:0.0100
Episode:6493 meanR:53.3800 R:48.0000 rate:0.0960 aloss:1.1826 dloss:0.6818 aloss2:12.3423 exploreP:0.0100
Episode:6494 meanR:54.0000 R:98.0000 rate:0.1960 aloss:1.1895 dloss:0.6713 aloss2:12.2181 exploreP:0.0100
Episode:6495 meanR:54.1200 R:40.0000 rate:0.0800 aloss:1.1889 dloss:0.6697 aloss2:12.1045 exploreP:0.0100
Episode:6496 meanR:54.3300 R:73.0000 rate:0.1460 aloss:1.1957 dloss:0.6892 aloss2:12.1927 exploreP:0.0100
Episode:6497 meanR:54.5200 R:60.0000 rate:0.1200 aloss:1.1860 dloss:0.6949 aloss2:12.1330 exploreP:0.0100
Episode:6498 meanR:54.5800 R:46.0000 rate:0.0

Episode:6567 meanR:63.0900 R:50.0000 rate:0.1000 aloss:1.1677 dloss:0.6542 aloss2:12.4355 exploreP:0.0100
Episode:6568 meanR:64.3400 R:164.0000 rate:0.3280 aloss:1.1620 dloss:0.6998 aloss2:12.6531 exploreP:0.0100
Episode:6569 meanR:64.3500 R:62.0000 rate:0.1240 aloss:1.1671 dloss:0.6829 aloss2:12.4279 exploreP:0.0100
Episode:6570 meanR:64.6600 R:68.0000 rate:0.1360 aloss:1.1585 dloss:0.7039 aloss2:12.4616 exploreP:0.0100
Episode:6571 meanR:69.2100 R:500.0000 rate:1.0000 aloss:1.1687 dloss:0.7088 aloss2:12.2988 exploreP:0.0100
Episode:6572 meanR:73.8000 R:500.0000 rate:1.0000 aloss:1.1545 dloss:0.7419 aloss2:12.3712 exploreP:0.0100
Episode:6573 meanR:78.3400 R:500.0000 rate:1.0000 aloss:1.1642 dloss:0.7144 aloss2:12.3907 exploreP:0.0100
Episode:6574 meanR:78.3700 R:61.0000 rate:0.1220 aloss:1.1514 dloss:0.7913 aloss2:12.3659 exploreP:0.0100
Episode:6575 meanR:78.9400 R:96.0000 rate:0.1920 aloss:1.1635 dloss:0.7280 aloss2:12.3046 exploreP:0.0100
Episode:6576 meanR:79.0800 R:42.0000 rate:

Episode:6645 meanR:95.7800 R:56.0000 rate:0.1120 aloss:1.1370 dloss:0.7794 aloss2:12.0327 exploreP:0.0100
Episode:6646 meanR:95.7600 R:63.0000 rate:0.1260 aloss:1.1449 dloss:0.7411 aloss2:12.0355 exploreP:0.0100
Episode:6647 meanR:95.9600 R:73.0000 rate:0.1460 aloss:1.1367 dloss:0.7231 aloss2:12.1448 exploreP:0.0100
Episode:6648 meanR:97.6500 R:296.0000 rate:0.5920 aloss:1.1431 dloss:0.7838 aloss2:11.9757 exploreP:0.0100
Episode:6649 meanR:97.9400 R:92.0000 rate:0.1840 aloss:1.1412 dloss:0.7792 aloss2:11.9738 exploreP:0.0100
Episode:6650 meanR:97.9400 R:46.0000 rate:0.0920 aloss:1.1301 dloss:0.7669 aloss2:12.0004 exploreP:0.0100
Episode:6651 meanR:97.9700 R:42.0000 rate:0.0840 aloss:1.1358 dloss:0.9068 aloss2:12.0955 exploreP:0.0100
Episode:6652 meanR:98.0400 R:54.0000 rate:0.1080 aloss:1.1362 dloss:0.8072 aloss2:11.8431 exploreP:0.0100
Episode:6653 meanR:97.9900 R:48.0000 rate:0.0960 aloss:1.1388 dloss:0.7791 aloss2:11.9435 exploreP:0.0100
Episode:6654 meanR:97.7900 R:58.0000 rate:0.1

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.